# Вычислительная математика. Лабораторная работа №2

In [94]:
import numpy as np

Проинициализируем матрицу. Я выбрал номер г.

In [95]:
n = 100

A = np.zeros((n, n))
b = np.zeros(n)

# Первая строка
A[0, :] = 1
b[0] = 100

# Строки 2-99
for i in range(1, n-1):
    A[i, i-1] = 1
    A[i, i] = 10
    A[i, i+1] = 1
    b[i] = 100 - i  # Уменьшение на 1 для каждой строки

# Строка 100
A[n-1, n-2] = 1
A[n-1, n-1] = 1
b[n-1] = 1

print("Матрица A:")
print(A)
print("Вектор b:")
print(b)


Матрица A:
[[ 1.  1.  1. ...  1.  1.  1.]
 [ 1. 10.  1. ...  0.  0.  0.]
 [ 0.  1. 10. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 10.  1.  0.]
 [ 0.  0.  0. ...  1. 10.  1.]
 [ 0.  0.  0. ...  0.  1.  1.]]
Вектор b:
[100.  99.  98.  97.  96.  95.  94.  93.  92.  91.  90.  89.  88.  87.
  86.  85.  84.  83.  82.  81.  80.  79.  78.  77.  76.  75.  74.  73.
  72.  71.  70.  69.  68.  67.  66.  65.  64.  63.  62.  61.  60.  59.
  58.  57.  56.  55.  54.  53.  52.  51.  50.  49.  48.  47.  46.  45.
  44.  43.  42.  41.  40.  39.  38.  37.  36.  35.  34.  33.  32.  31.
  30.  29.  28.  27.  26.  25.  24.  23.  22.  21.  20.  19.  18.  17.
  16.  15.  14.  13.  12.  11.  10.   9.   8.   7.   6.   5.   4.   3.
   2.   1.]


## 1. Метод Гаусса с выбором главного элемента

In [96]:
def gauss_with_pivoting(A, b):
    n = len(b)

    for i in range(n):
        max_row = np.argmax(np.abs(A[i:, i])) + i # Главный элемент
        # Смена строк местами
        A[[i, max_row]] = A[[max_row, i]]
        b[[i, max_row]] = b[[max_row, i]]
        
        # Приводим текущую строку к нужному виду
        for j in range(i+1, n):
            factor = A[j][i] / A[i][i]
            A[j] = A[j] - factor * A[i]
            b[j] = b[j] - factor * b[i]
    
    # Обратный ход
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i][i]
    
    return x

In [97]:
solution = gauss_with_pivoting(A.copy(), b.copy())

print("Решение системы:")
print(solution)

Решение системы:
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.3333

## 2. Метод LU-разложения

In [98]:
def lu_decomposition(A):
    n = len(A)
    L = np.zeros_like(A)
    U = np.zeros_like(A)
    
    # LU разложение без выбора главного элемента
    for i in range(n):
        # Заполняем матрицу U
        for k in range(i, n):
            U[i, k] = A[i, k] - sum(L[i, j] * U[j, k] for j in range(i))
        
        # Заполняем матрицу L
        for k in range(i, n):
            if i == k:
                L[i, i] = 1  # Диагональные элементы равны 1
            else:
                L[k, i] = (A[k, i] - sum(L[k, j] * U[j, i] for j in range(i))) / U[i, i]
    
    return L, U

def forward_substitution(L, b):
    n = len(b)
    y = np.zeros_like(b)
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i]))
    return y

def backward_substitution(U, y):
    n = len(y)
    x = np.zeros_like(y)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

In [99]:
L, U = lu_decomposition(A.copy())

y = forward_substitution(L, b.copy())

solution = backward_substitution(U, y)

print("Решение системы:")
print(solution)

Решение системы:
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.3333

## 3. Метод Якоби

In [100]:
def jacobi_method(A, b, tol=1e-10, max_iterations=1000):
    n = len(b)
    x = np.zeros_like(b)  # начальное приближение
    x_new = np.zeros_like(x)
    
    for iteration in range(max_iterations):
        for i in range(n):
            s = np.dot(A[i, :i], x[:i]) + np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - s) / A[i, i]
        
        # Проверяем условие сходимости
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            break
        x = x_new.copy()
    
    return x

In [101]:
solution = jacobi_method(A.copy(), b.copy())

print("Решение системы:")
print(solution)

Решение системы:
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.3333

## 4. Метод Зейделя

In [102]:
import numpy as np

def seidel_method(A, b, tol=1e-10, max_iterations=1000):
    n = len(b)
    x = np.zeros_like(b)  # начальное приближение
    
    for iteration in range(max_iterations):
        x_new = np.zeros_like(x)
        for i in range(n):
            s1 = np.dot(A[i, :i], x_new[:i])  # Сумма по уже обновленным элементам
            s2 = np.dot(A[i, i+1:], x[i+1:])  # Сумма по не обновленным элементам
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        
        # Проверяем условие сходимости
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            break
        x = x_new.copy()
    
    return x

In [103]:
solution = seidel_method(A.copy(), b.copy())

print("Решение системы:")
print(solution)


Решение системы:
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.3333

## 5. Метод верхней релаксации

In [104]:
def relaxation_method(A, b, tau=1.25, max_iterations=10000, tolerance=1e-10):
    n = len(b)
    x = np.zeros(n)  # Начальное приближение

    for iteration in range(max_iterations):
        x_old = np.copy(x)

        for i in range(n):
            sum1 = np.dot(A[i, :i], x[:i])
            sum2 = np.dot(A[i, i+1:], x_old[i+1:])
            x[i] = (1 - tau) * x_old[i] + (tau/ A[i, i]) * (b[i] - sum1 - sum2)

        if np.linalg.norm(x - x_old, ord=np.inf) < tolerance:
            return x


In [105]:
solution = relaxation_method(A.copy(), b.copy(), tau=1.25)

print("Решение системы:")
print(solution)

Решение системы:
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.3333

## 6. Метод градиентного спуска

In [106]:
def gradient_descent(A, b, tol=1e-20, max_iterations=1000):
    n = len(b)
    x = np.zeros_like(b) 
    alpha = 0.008  # Скорость обучения

    for iteration in range(max_iterations):
        r = np.dot(A, x) - b  # Невязка
        
        gradient = np.dot(A.T, r)  # Градиент функции

        x_new = x - alpha * gradient
        
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            break
        
        x = x_new.copy()

    return x

In [107]:
solution = gradient_descent(A.copy(), b.copy())

print("Решение системы:")
print(solution)

Решение системы:
[-3.09317878e+02  4.02407878e+01  4.85829866e+00  8.33758566e+00
  7.89480320e+00  7.84772468e+00  7.76073581e+00  7.67777093e+00
  7.59440047e+00  7.51107088e+00  7.42773717e+00  7.34440387e+00
  7.26107054e+00  7.17773720e+00  7.09440387e+00  7.01107054e+00
  6.92773720e+00  6.84440387e+00  6.76107054e+00  6.67773720e+00
  6.59440387e+00  6.51107054e+00  6.42773720e+00  6.34440387e+00
  6.26107054e+00  6.17773720e+00  6.09440387e+00  6.01107054e+00
  5.92773720e+00  5.84440387e+00  5.76107054e+00  5.67773720e+00
  5.59440387e+00  5.51107054e+00  5.42773720e+00  5.34440387e+00
  5.26107054e+00  5.17773720e+00  5.09440387e+00  5.01107054e+00
  4.92773720e+00  4.84440387e+00  4.76107054e+00  4.67773720e+00
  4.59440387e+00  4.51107054e+00  4.42773720e+00  4.34440387e+00
  4.26107054e+00  4.17773720e+00  4.09440387e+00  4.01107054e+00
  3.92773720e+00  3.84440387e+00  3.76107054e+00  3.67773720e+00
  3.59440387e+00  3.51107054e+00  3.42773720e+00  3.34440387e+00
  3.2610

Различие в ответе я связываю с подбором параметра alpha - он не адаптивен в этом методе, поэтому решение колеблется около правильного в зависимости от подбора alpha

## 7. Метод минимальных невязок

In [108]:
def minimum_residual_method(A, b, tol=1e-10, max_iterations=1000):
    n = len(b)
    x = np.zeros_like(b) 

    for iteration in range(max_iterations):
        # Невязку
        r = b - np.dot(A, x)  # Остаток

        # Оптимальный шаг
        Ar = np.dot(A, r)
        alpha = np.dot(r, r) / np.dot(Ar, r)

        x_new = x + alpha * r
        
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            break
        
        x = x_new.copy()

    return x

In [109]:
solution = minimum_residual_method(A.copy(), b.copy())

print("Решение системы:")
print(solution)

Решение системы:
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.3333